Версии:
  - sqlalchemy 1.4.22
  - pandas 1.4.2

In [1]:
import sqlalchemy
import sqlalchemy.orm
import pandas as pd

Параметры подключения (сервер, БД, драйвер)

In [2]:
SERVER = 'KONSTANTIN\SQLEXPRESS'
DB = 'testdb'
DRIVER = 'SQL Server'

In [3]:
conn = sqlalchemy.create_engine(
    f'mssql+pyodbc://@{SERVER}/{DB}?trusted_connection=yes&driver={DRIVER}')
session = sqlalchemy.orm.Session(conn)

Создание таблицы

In [4]:
with session.begin():
    conn.execute('''
            CREATE TABLE T_CONTRACTOR_SHERULER (
                ID_NAME INT IDENTITY(1, 1) NOT NULL PRIMARY KEY,
                NAME NVARCHAR(30) NOT NULL,
                SHEDULE NVARCHAR(20) NOT NULL,
                DATE_BEGIN DATETIME NOT NULL,
                DATE_END DATETIME NOT NULL,

                CONSTRAINT uq UNIQUE (NAME, SHEDULE),
                CONSTRAINT dt CHECK (DATE_BEGIN <= DATE_END)
                )
                   ''')

Мой экземпляр БД использует американский формат даты, фикс

In [5]:
with session.begin():
    conn.execute('''SET DATEFORMAT dmy;''')

Чтение и заливка данных

In [6]:
df = pd.read_csv(r'.\sql\shedulers.csv',
                 sep=';',
                 encoding='cp1251'
                )
df.columns = ['NAME', 'SHEDULE', 'DATE_BEGIN', 'DATE_END']
_ = df.to_sql('T_CONTRACTOR_SHERULER', con=conn, if_exists='append', index=False)